In [1]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 200)

In [2]:
df = pl.read_csv("/home/surayuth/her2/extracted_features/linefeat|scale_0.25|minhole_10|maxhole_1000|maxres_0.25.csv")

In [4]:
df \
    .group_by("case") \
    .agg(
        pl.col("n").mean(),
        pl.len().alias("count"),
        pl.when(pl.col("deg") == 0).then(pl.lit(1)).sum().alias("num0"),
        pl.when(pl.col("deg") == 1).then(pl.lit(1)).sum().alias("num1"),
        pl.when(pl.col("deg") > 1).then(pl.lit(1)).sum().alias("num>1"),
        pl.when(pl.col("deg") == 1).then(pl.col("param_v")).mean().alias("avg_param_v1"),
        pl.col("mean_y").mean(),
        pl.col("ihc_score").first(),
        pl.col("label").first()
    ) \
    .sort("ihc_score", "avg_param_v1", "case") \
    .to_pandas()

,case,n,count,num0,num1,num>1,avg_param_v1,mean_y,ihc_score,label
0,11 Oct HER2 0 case 1,0.000000,70,70,0,0,NaN,0.000000,0,0
1,11 Oct HER2 0 case 2,0.000000,30,30,0,0,NaN,0.000000,0,0
2,11 Oct HER2 0 case 3,0.000000,28,28,0,0,NaN,0.000000,0,0
3,13 Sep HER2 0 case 1,0.000000,48,48,0,0,NaN,0.000000,0,0
4,13 Sep HER2 0 case 2,0.000000,44,44,0,0,NaN,0.000000,0,0
5,21 Sep HER2 0 case 1,0.000000,55,55,0,0,NaN,0.000000,0,0
6,21 Sep HER2 score 0 case 2,0.000000,26,26,0,0,NaN,0.000000,0,0
7,21 Sep HER2 score 0 case 3,0.000000,31,31,0,0,NaN,0.000000,0,0
8,3 Sep HER2 0 Case 1,0.000000,56,56,0,0,NaN,0.000000,0,0
9,3 Sep HER2 0 case 2,0.000000,72,72,0,0,NaN,0.000000,0,0
